# Fully connected feedforward network implementing a loss mask

In [ ]:
import torchmetrics.classification
from torch import cuda
assert cuda.is_available()
assert cuda.device_count() > 0

In [ ]:
print(cuda.get_device_name(cuda.current_device()))


NVIDIA GeForce RTX 3060 Ti


In [ ]:
import torch
import torch.nn as nn

In [ ]:
DEVICE = torch.device("cuda")
SEED = 76436278

torch.manual_seed(SEED)

### Load the Dataset

In [ ]:
from src.maldi2resistance.data.driams import Driams

driams = Driams(
    root_dir="/home/jan/Uni/master/data/Driams",
)
umg = Driams(
    root_dir="/home/jan/Uni/master/data/Driams",
    bin_size=1,
    sites=["UMG"],
    years=[2020,2021],
    antibiotics=driams.selected_antibiotics,
)

driams.loading_type = "memory"
umg.loading_type = "memory"

driams

/home/jan/Uni/master/src/maldi2resistance/data/driams.py:86: UserWarning: Metafile for Site (DRIAMS-B) and Year (2015) combination not found!
  warnings.warn(
/home/jan/Uni/master/src/maldi2resistance/data/driams.py:86: UserWarning: Metafile for Site (DRIAMS-B) and Year (2016) combination not found!
  warnings.warn(
/home/jan/Uni/master/src/maldi2resistance/data/driams.py:86: UserWarning: Metafile for Site (DRIAMS-B) and Year (2017) combination not found!
  warnings.warn(
/home/jan/Uni/master/src/maldi2resistance/data/driams.py:86: UserWarning: Metafile for Site (DRIAMS-C) and Year (2015) combination not found!
  warnings.warn(
/home/jan/Uni/master/src/maldi2resistance/data/driams.py:86: UserWarning: Metafile for Site (DRIAMS-C) and Year (2016) combination not found!
  warnings.warn(
/home/jan/Uni/master/src/maldi2resistance/data/driams.py:86: UserWarning: Metafile for Site (DRIAMS-C) and Year (2017) combination not found!
  warnings.warn(
/home/jan/Uni/master/src/maldi2resistance/data

Loading Spectra into Memory:   0%|          | 0/55780 [00:00<?, ?it/s]

Loading Spectra into Memory:   0%|          | 0/73745 [00:00<?, ?it/s]

Antibiotic:,Amikacin,Amoxicillin-Clavulanic acid,Ampicillin,Ampicillin-Amoxicillin,Aztreonam,Benzylpenicillin,Cefazolin,Cefepime,Cefpodoxime,Ceftazidime,Ceftriaxone,Cefuroxime,Ciprofloxacin,Clarithromycin,Clindamycin,Colistin,Cotrimoxazole,Ertapenem,Erythromycin,Fosfomycin,Fosfomycin-Trometamol,Fusidic acid,Gentamicin,Imipenem,Levofloxacin,Meropenem,Mupirocin,Nitrofurantoin,Norfloxacin,Oxacillin,Penicillin,Piperacillin-Tazobactam,Polymyxin B,Rifampicin,Teicoplanin,Tetracycline,Tobramycin,Vancomycin
Number resistant:,1068,13366,8578,21966,628,618,4223,7383,2338,3470,8659,5855,9338,310,4381,2874,7405,427,5468,2303,1326,3620,3481,7560,4217,5194,570,1271,1205,5537,12431,7616,486,580,244,3534,1707,227
Number susceptible:,20941,24992,4194,4905,456,457,5813,31567,4382,24566,28464,8368,36822,1262,9841,15784,24590,21740,9044,10184,4803,8498,22662,31717,17989,27228,4656,3603,7031,7740,4286,31308,2305,14964,8486,10376,16809,20540
Number data points:,22009,38358,12772,26871,1084,1075,10036,38950,6720,28036,37123,14223,46160,1572,14222,18658,31995,22167,14512,12487,6129,12118,26143,39277,22206,32422,5226,4874,8236,13277,16717,38924,2791,15544,8730,13910,18516,20767


In [ ]:
len(driams.label_stats.columns)

38

In [ ]:
import copy
from maldi2resistance.model.singleBranchMlp import SingleBranchMLP

model = SingleBranchMLP( input_dim= 18000, output_dim= len(driams.selected_antibiotics))
model = model.to(DEVICE)

model_state = copy.deepcopy(model.state_dict()) 

In [ ]:
from torchinfo import summary

print(summary(model, input_data=torch.zeros((1,18000)).to(DEVICE)))

Layer (type:depth-idx)                   Output Shape              Param #
SingleBranchMLP                          [1, 38]                   --
├─Sequential: 1-1                        [1, 38]                   --
│    └─Linear: 2-1                       [1, 512]                  9,216,512
│    └─GELU: 2-2                         [1, 512]                  --
│    └─Dropout: 2-3                      [1, 512]                  --
│    └─LayerNorm: 2-4                    [1, 512]                  1,024
│    └─Linear: 2-5                       [1, 256]                  131,328
│    └─GELU: 2-6                         [1, 256]                  --
│    └─Dropout: 2-7                      [1, 256]                  --
│    └─LayerNorm: 2-8                    [1, 256]                  512
│    └─Linear: 2-9                       [1, 128]                  32,896
│    └─GELU: 2-10                        [1, 128]                  --
│    └─Dropout: 2-11                     [1, 128]                

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
from maldi2resistance.metric.PrecisionRecall import MultiLabelPRNan
from pathlib import Path
from maldi2resistance.metric.ROC import MultiLabelRocNan
from maldi2resistance.loss.maskedLoss import MaskedBCE
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR

print("Start training ...")
model.train()

batch_size = 128
fig_path = Path("./kfold/figures")
fig_path.mkdir(parents=True, exist_ok=True)

loss_per_batch = []

class_weights_negative = torch.tensor((1 - ((driams.label_stats.loc["negative"] +umg.label_stats.loc["negative"]) / (driams.label_stats.loc["n_sum"] +umg.label_stats.loc["n_sum"]))).values, device=DEVICE)
class_weights_positive = torch.tensor((1 - ((driams.label_stats.loc["positive"] + umg.label_stats.loc["positive"]) / (driams.label_stats.loc["n_sum"]+umg.label_stats.loc["n_sum"]))).values, device=DEVICE)

criterion = MaskedBCE()

gen = torch.Generator()

for fold, (train_data, test_data) in enumerate(driams.getK_fold(n_splits=5, shuffle=True, random_state= SEED)):
    train_data_umg = torch.utils.data.ConcatDataset([train_data, umg])
    
    train_loader = DataLoader(train_data_umg, batch_size=batch_size, shuffle=True, drop_last=True, generator= gen.manual_seed(SEED))
    test_loader = DataLoader(test_data, batch_size=len(test_data), shuffle=True, drop_last=True, generator= gen.manual_seed(SEED))
    
    model.load_state_dict(model_state)
    model.train()
    
    optimizer = Adam(model.parameters(), lr=1e-3, amsgrad = True)
    scheduler = StepLR(optimizer, step_size=10, gamma=0.5)

    for epoch in tqdm(range(30)):
        overall_loss = 0
        
        for batch_idx, (x, y) in enumerate(train_loader):
    
            x = x.to(DEVICE)
            y = y.to(DEVICE)
            
            optimizer.zero_grad()
    
            output = model(x)
        
            loss = criterion(output, y)
            current_loss_value = loss.item()
            loss_per_batch.append(current_loss_value)
            
            overall_loss += current_loss_value
            
            loss.backward()
            optimizer.step()
    
        scheduler.step()
        with tqdm.external_write_mode():
            print(f"\tAverage Loss: {overall_loss / (batch_idx*batch_size):.6f} \tLearning rate: {scheduler.get_last_lr()[0]:.6f}")
    
    
    print(f"Finished Fold {fold}")
    
    model.eval()
    
    DEVICE = torch.device("cpu")
    test_features, test_labels = next(iter(test_loader))
    test_features = test_features.to(DEVICE)
    test_labels = test_labels.to(DEVICE)
    model = model.to(DEVICE)
    
    ml_roc = MultiLabelRocNan()
    output = model(test_features)
    
    
    ml_roc.compute(output,test_labels,driams.selected_antibiotics, create_csv=f"./kfold/csv/fold-{fold}_ROC.csv")
    fig_, ax_ = ml_roc()
    
    plt.savefig(fig_path / f"fold-{fold}_ROC.png", transparent=True, format= "png", bbox_inches = "tight")
    plt.close()
    
    ml_pr = MultiLabelPRNan()
    ml_pr.compute(output,test_labels,driams.selected_antibiotics, create_csv=f"./kfold/csv/fold-{fold}_PrecisionRecall.csv")
    
    fig_, ax_ = ml_pr()
    
    plt.savefig(fig_path / f"fold-{fold}_PrecisionRecall.png", transparent=True, format= "png", bbox_inches = "tight")
    plt.close()
    
    DEVICE = torch.device("cuda")
    model = model.to(DEVICE)

Start training ...


  0%|          | 0/30 [00:00<?, ?it/s]

	Average Loss: 0.000720 	Learning rate: 0.001000
	Average Loss: 0.000608 	Learning rate: 0.001000
	Average Loss: 0.000572 	Learning rate: 0.001000
	Average Loss: 0.000554 	Learning rate: 0.001000
	Average Loss: 0.000537 	Learning rate: 0.001000
	Average Loss: 0.000527 	Learning rate: 0.001000
	Average Loss: 0.000518 	Learning rate: 0.001000
	Average Loss: 0.000509 	Learning rate: 0.001000
	Average Loss: 0.000502 	Learning rate: 0.001000
	Average Loss: 0.000497 	Learning rate: 0.000500
	Average Loss: 0.000473 	Learning rate: 0.000500
	Average Loss: 0.000467 	Learning rate: 0.000500
	Average Loss: 0.000463 	Learning rate: 0.000500
	Average Loss: 0.000458 	Learning rate: 0.000500
	Average Loss: 0.000455 	Learning rate: 0.000500
	Average Loss: 0.000450 	Learning rate: 0.000500
	Average Loss: 0.000447 	Learning rate: 0.000500
	Average Loss: 0.000443 	Learning rate: 0.000500
	Average Loss: 0.000441 	Learning rate: 0.000500
	Average Loss: 0.000438 	Learning rate: 0.000250
	Average Loss: 0.000

  0%|          | 0/30 [00:00<?, ?it/s]

	Average Loss: 0.000723 	Learning rate: 0.001000
	Average Loss: 0.000604 	Learning rate: 0.001000
	Average Loss: 0.000571 	Learning rate: 0.001000
	Average Loss: 0.000551 	Learning rate: 0.001000
	Average Loss: 0.000538 	Learning rate: 0.001000
	Average Loss: 0.000527 	Learning rate: 0.001000
	Average Loss: 0.000516 	Learning rate: 0.001000
	Average Loss: 0.000509 	Learning rate: 0.001000
	Average Loss: 0.000503 	Learning rate: 0.001000
	Average Loss: 0.000496 	Learning rate: 0.000500
	Average Loss: 0.000473 	Learning rate: 0.000500
	Average Loss: 0.000467 	Learning rate: 0.000500
	Average Loss: 0.000462 	Learning rate: 0.000500
	Average Loss: 0.000459 	Learning rate: 0.000500
	Average Loss: 0.000455 	Learning rate: 0.000500
	Average Loss: 0.000451 	Learning rate: 0.000500
	Average Loss: 0.000447 	Learning rate: 0.000500
	Average Loss: 0.000444 	Learning rate: 0.000500
	Average Loss: 0.000441 	Learning rate: 0.000500
	Average Loss: 0.000438 	Learning rate: 0.000250
	Average Loss: 0.000

...

	Average Loss: 0.000722 	Learning rate: 0.001000
	Average Loss: 0.000609 	Learning rate: 0.001000
	Average Loss: 0.000575 	Learning rate: 0.001000
	Average Loss: 0.000553 	Learning rate: 0.001000
	Average Loss: 0.000542 	Learning rate: 0.001000
	Average Loss: 0.000528 	Learning rate: 0.001000
	Average Loss: 0.000520 	Learning rate: 0.001000
	Average Loss: 0.000511 	Learning rate: 0.001000
	Average Loss: 0.000504 	Learning rate: 0.001000
	Average Loss: 0.000498 	Learning rate: 0.000500
	Average Loss: 0.000474 	Learning rate: 0.000500
	Average Loss: 0.000469 	Learning rate: 0.000500
	Average Loss: 0.000464 	Learning rate: 0.000500
	Average Loss: 0.000461 	Learning rate: 0.000500
	Average Loss: 0.000457 	Learning rate: 0.000500
	Average Loss: 0.000453 	Learning rate: 0.000500
	Average Loss: 0.000450 	Learning rate: 0.000500
	Average Loss: 0.000446 	Learning rate: 0.000500
	Average Loss: 0.000442 	Learning rate: 0.000500
	Average Loss: 0.000439 	Learning rate: 0.000250
	Average Loss: 0.000

  0%|          | 0/30 [00:00<?, ?it/s]

	Average Loss: 0.000719 	Learning rate: 0.001000
	Average Loss: 0.000605 	Learning rate: 0.001000
	Average Loss: 0.000571 	Learning rate: 0.001000
	Average Loss: 0.000550 	Learning rate: 0.001000
	Average Loss: 0.000537 	Learning rate: 0.001000
	Average Loss: 0.000527 	Learning rate: 0.001000
	Average Loss: 0.000517 	Learning rate: 0.001000
	Average Loss: 0.000508 	Learning rate: 0.001000
	Average Loss: 0.000501 	Learning rate: 0.001000
	Average Loss: 0.000497 	Learning rate: 0.000500
	Average Loss: 0.000473 	Learning rate: 0.000500
	Average Loss: 0.000466 	Learning rate: 0.000500
	Average Loss: 0.000460 	Learning rate: 0.000500
	Average Loss: 0.000457 	Learning rate: 0.000500
	Average Loss: 0.000454 	Learning rate: 0.000500
	Average Loss: 0.000451 	Learning rate: 0.000500
	Average Loss: 0.000446 	Learning rate: 0.000500
	Average Loss: 0.000443 	Learning rate: 0.000500
	Average Loss: 0.000439 	Learning rate: 0.000500
	Average Loss: 0.000436 	Learning rate: 0.000250
	Average Loss: 0.000

  0%|          | 0/30 [00:00<?, ?it/s]

	Average Loss: 0.000722 	Learning rate: 0.001000
	Average Loss: 0.000606 	Learning rate: 0.001000
	Average Loss: 0.000574 	Learning rate: 0.001000
	Average Loss: 0.000555 	Learning rate: 0.001000
	Average Loss: 0.000540 	Learning rate: 0.001000
	Average Loss: 0.000527 	Learning rate: 0.001000
	Average Loss: 0.000521 	Learning rate: 0.001000
	Average Loss: 0.000512 	Learning rate: 0.001000
	Average Loss: 0.000505 	Learning rate: 0.001000
	Average Loss: 0.000498 	Learning rate: 0.000500
	Average Loss: 0.000475 	Learning rate: 0.000500
	Average Loss: 0.000470 	Learning rate: 0.000500
	Average Loss: 0.000464 	Learning rate: 0.000500
	Average Loss: 0.000460 	Learning rate: 0.000500
	Average Loss: 0.000457 	Learning rate: 0.000500
	Average Loss: 0.000453 	Learning rate: 0.000500
	Average Loss: 0.000449 	Learning rate: 0.000500
	Average Loss: 0.000446 	Learning rate: 0.000500
	Average Loss: 0.000443 	Learning rate: 0.000500
	Average Loss: 0.000440 	Learning rate: 0.000250
	Average Loss: 0.000

In [ ]:
import pandas
micro = []
macro = []

for fold in range(0,5):
    csv = pandas.read_csv(f"./kfold/csv/fold-{fold}_ROC.csv")
    micro.append(csv[csv["class"] == "micro"]["ROCAUC"])
    macro.append(csv[csv["class"] == "macro"]["ROCAUC"])

In [ ]:
import numpy as np

print(f"Mean\t: {np.mean(micro)}")
print(f" SD \t: {np.std(micro) }")

Mean	: 0.9345815062522889
 SD 	: 0.0012093686266301086


In [ ]:
print(f"Mean\t: {np.mean(macro)}")
print(f" SD \t: {np.std(macro) }")

Mean	: 0.8988297688333612
 SD 	: 0.0011125938688271498


In [ ]:
import pandas
micro = []
macro = []

for fold in range(0,5):
    csv = pandas.read_csv(f"./kfold/csv/fold-{fold}_PrecisionRecall.csv")
    micro.append(csv[csv["class"] == "micro"]["PrecisionRecallAUC"])
    macro.append(csv[csv["class"] == "macro"]["PrecisionRecallAUC"])

In [ ]:
import numpy as np

print(f"Mean\t: {np.mean(micro)}")
print(f" SD \t: {np.std(micro) }")

Mean	: 0.865748918056488
 SD 	: 0.006942192350804705


In [ ]:
print(f"Mean\t: {np.mean(macro)}")
print(f" SD \t: {np.std(macro) }")

Mean	: 0.7538028109230492
 SD 	: 0.0070839035312569906
